# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [5]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

Score: 0.0


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [7]:
import tqdm

from unityagents import UnityEnvironment
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
# Get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
    
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of actions
num_actions = brain.vector_action_space_size
print('Number of actions:', num_actions)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of actions: 4
States look like: [0.         0.         1.         0.         0.16101955 1.
 0.         0.         0.         0.04571758 1.         0.
 0.         0.         0.2937662  0.         0.         1.
 0.         0.14386636 0.         0.         1.         0.
 0.16776823 1.         0.         0.         0.         0.04420976
 1.         0.         0.         0.         0.05423063 0.
 0.        ]
States have length: 37


In [42]:
class Q(nn.Module):
    
    def __init__(self, state_size, action_size):
        super(Q, self).__init__()
        
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, action_size)
        
        self.fc1.weight.data.normal_(std=1/state_size)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.normal_(std=1/256)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.normal_(std=1/128)
        self.fc2.bias.data.zero_()
        
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        return x
    
    def td_target(self, rewards, next_states, dones, gamma):
        device = self.get_device()
        next_states = torch.as_tensor(next_states, dtype=torch.float32).to(device)
        rewards = torch.as_tensor(rewards, dtype=torch.float32).to(device)
        dones = torch.as_tensor(dones.astype(int), dtype=torch.float32).to(device)
        max_q, _ = torch.max(self.forward(next_states), dim=-1, keepdim=True)
        y = rewards + (1 - dones) * gamma * max_q
        return y
        
    def loss(self, states, actions, td_targets):
        device = self.get_device()
        states = torch.as_tensor(states, dtype=torch.float32).to(device)
        actions = torch.as_tensor(actions, dtype=torch.long).to(device)
        q = self.forward(states).gather(1, actions.view(-1, 1))
        loss = F.mse_loss(q, td_targets.to(device))
        return loss
    
    def get_device(self):
        return next(self.parameters()).device
    
    def action_values(self, state):
        state = torch.as_tensor(state, dtype=torch.float32).to(self.get_device())
        values = self.forward(state)
        return values.detach().cpu().numpy()
    
    def pick_action(self, state):
        state = torch.as_tensor(state, dtype=torch.float32).to(self.get_device())
        action = torch.argmax(self.forward(state), dim=-1, keepdim=True)
        return action.detach().cpu().numpy()
    
    def copy(self, other):
        self.load_state_dict(other.state_dict())
    
q = Q(state_size, action_size)
print(q)

env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]
action_values = q.action_values(state)
print(action_values)

states = [state, state]
action_values = q.action_values(states)
print(action_values)

q2 = Q(state_size, action_size)
q2.copy(q)
assert np.array_equal(q.fc1.weight.data.detach().numpy(), q2.fc1.weight.data.detach().numpy())

print('=' * 50)

states = np.zeros([2, state_size])
actions = np.zeros([2, 1], dtype=int)
rewards = np.zeros([2, 1])
next_states = np.zeros([2, state_size])
dones = np.full([2, 1], False)

q.to(device)

env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]

for i in range(2):
    action = q.pick_action(state)
    env_info = env.step(action)[brain_name]
    reward = env_info.rewards[0]
    next_state = env_info.vector_observations[0]
    done = env_info.local_done[0]
    states[i] = state
    actions[i] = action
    rewards[i] = reward
    next_states[i] = next_state
    dones[i] = done
    state = next_state

print(states[:, :5])
print(actions)
print(rewards)
print(next_states[:, :5])
print(dones)

optimizer = torch.optim.Adam(q.parameters())
optimizer.zero_grad()
y = q.td_target(rewards, next_states, dones, gamma=1.0)
loss = q.loss(states, actions, y)
loss.backward()
optimizer.step()

Q(
  (fc1): Linear(in_features=37, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=4, bias=True)
)
[ 0.06685083  0.03782009  0.00553394 -0.03237629]
[[ 0.06685083  0.03782009  0.00553394 -0.03237629]
 [ 0.06685083  0.03782009  0.00553394 -0.03237629]]
[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]]
[[0]
 [0]]
[[0.]
 [0.]]
[[0.         0.         0.         1.         0.        ]
 [0.         0.         1.         0.         0.35670558]]
[[False]
 [False]]


In [12]:
def egreedy(state, q_model, epsilon=0.1):
    if np.random.random() >= epsilon:
        return int(np.argmax(q_model.action_values(state)))
    else:
        return np.random.randint(action_size)

q_model = Q(state_size, action_size)
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]
action = egreedy(state, q_model, epsilon=0.)
print(action, type(action))
env_info = env.step(action)[brain_name]

0 <class 'int'>


In [13]:
class ReplayMemory:
    def __init__(self, state_shape, action_shape, capacity=2**20):
        self.capacity = capacity
        self.states = np.zeros([capacity, *state_shape])
        self.actions = np.zeros([capacity, *action_shape], dtype=int)
        self.rewards = np.zeros([capacity, 1])
        self.next_states = np.zeros([capacity, *state_shape])
        self.dones = np.full([capacity, 1], False)
        self.cursor = 0
        self.full = False
        
    def store(self, state, action, reward, next_state, done):
        self.states[self.cursor] = state
        self.actions[self.cursor] = action
        self.rewards[self.cursor] = reward
        self.next_states[self.cursor] = next_state
        self.dones[self.cursor] = done
        self.cursor = (self.cursor + 1) % self.capacity
        if not self.full and self.cursor == 0:
            self.full = True
            
    def sample(self, n_samples):
        if not self.full and self.cursor < n_samples:
            return [], [], [], [], []
        
        end_index = self.cursor if not self.full else self.capacity
        samples = np.random.choice(end_index, size=n_samples)
        return (
            self.states[samples],
            self.actions[samples],
            self.rewards[samples],
            self.next_states[samples],
            self.dones[samples])

q = Q(state_size, action_size)
replay = ReplayMemory([state_size], [1], capacity=5)
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.vector_observations[0]

for _ in range(8):
    action = egreedy(state, q, epsilon=0.)
    env_info = env.step(action)[brain_name]
    reward = env_info.rewards[0]
    next_state = env_info.vector_observations[0]
    done = env_info.local_done[0]
    replay.store(state, action, reward, next_state, done)
    state = next_state
    
samples = replay.sample(1)
print("samples:\n", samples)

print("states:\n", replay.states[:2, :5])
print("next_states:\n", replay.next_states[:2, :5])

samples:
 (array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.97567415e-01, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.71893001e-01, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.21931839e-01, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.09156588e-01,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        9.05856907e-01, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 3.39805841e-01, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 4.63989139e-01, 4.76837158e-07,
        1.16585569e+01]]), array([[0]]), array([[0.]]), array([[0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        4.65226740e-01, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.46574378e-01, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 3.09724241e-01, 1.00000000e+0

### TODO

  * target update interval
  * step skip interval
  * epsilon annealing

In [44]:
import sys

def train(env, num_episodes=1, alpha=1e-4, batch_size=32,
          epsilon=0.1, gamma=.99,
          target_update_interval=10**4,
          episode_scores=None,
          device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
    target_model = Q(state_size, action_size)
    behavior_model = Q(state_size, action_size).to(device)
    behavior_model.copy(target_model)
    target_model.eval()
    
    optimizer = torch.optim.Adam(behavior_model.parameters(), lr=alpha)

    def policy(state):
        return egreedy(state, behavior_model, epsilon)
    
    replay = ReplayMemory([state_size], [1])
    i_step = 0
    
    for i_episode in tqdm.trange(num_episodes, ncols=50):
        env_info = env.reset(train_mode=True)[brain_name]
        state = env_info.vector_observations[0]
        done = env_info.local_done[0]
        score = 0
        
        while not done:
            action = policy(state)
            env_info = env.step(action)[brain_name]
            next_state = env_info.vector_observations[0]
            reward = env_info.rewards[0]
            done = env_info.local_done[0]
            replay.store(state, action, reward, next_state, done)
            states, actions, rewards, next_states, dones = replay.sample(batch_size)
            if len(states) > 0:
                y = target_model.td_target(rewards, next_states, dones, gamma)
                behavior_model.train()
                optimizer.zero_grad()
                loss = behavior_model.loss(states, actions, y)
                loss.backward()
                optimizer.step()
                behavior_model.eval()
                
                if (i_step + 1) % target_update_interval == 0:
                    target_model.copy(behavior_model)
                i_step += 1

            score += reward
            state = next_state
        
        if episode_scores is not None:
            episode_scores.append(score)
        #sys.stdout.write('\rEpisode {}/{} Score: {}'.format(i_episode+1, num_episodes, score))
    #sys.stdout.write('\n')
    
    return target_model


for device1 in [torch.device('cuda'), torch.device('cpu')]:
    episode_scores = []
    train(env, num_episodes=10, batch_size=32, target_update_interval=5, episode_scores=episode_scores,
          device=device1)
    print(episode_scores)

100%|█████████████| 10/10 [00:50<00:00,  5.06s/it]


[0.0, 1.0, 4.0, 5.0, 3.0, 3.0, 1.0, 5.0, 8.0, 2.0]


100%|█████████████| 10/10 [00:23<00:00,  2.41s/it]


[1.0, 0.0, -1.0, 0.0, -1.0, 1.0, 3.0, 3.0, 4.0, 2.0]


In [ ]:
episode_scores = []
q_model = train(env, num_episodes=1000, batch_size=32, target_update_interval=100,
                episode_scores=episode_scores, device=torch.device('cpu'))

plt.plot(episode_scores)
plt.xlabel('episode')
plt.ylabel('score')
plt.title('Episode Score')
plt.show()


  8%|▊          | 76/1000 [02:46<31:38,  2.05s/it]

In [ ]:
%matplotlib inline
plt.plot(episode_scores)
plt.xlabel('episode')
plt.ylabel('score')
plt.title('Episode Score')
plt.show()

In [ ]:
torch.from_numpy(np.random.random([1, state_size]))

In [ ]:
random_state = torch.from_numpy(np.random.random([1, state_size])).float()
q_model.forward(random_state)

In [ ]:
aa = np.arange(10)
ii = np.array([1,3,9])
aa[ii]

In [ ]:
aa

In [ ]:
torch.rand(1, 3).size()

In [33]:
aa = np.array([True, False, True]).astype(int)
aa

array([1, 0, 1])

In [34]:
(np.logical_not(aa)) * np.array([1, 2, 3])

array([0, 2, 0])

In [18]:
torch.cuda.

1